In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
pd.set_option("display.max_columns",45)

**Key**

In [2]:
Patient="Casefortesting"

**Read CNV plot**

In [3]:
cnv=pd.read_excel(Patient+".seg_review.xlsx",index_col=None)
len(cnv)

1505

**Find unique samples**

In [4]:
samples=list(cnv.Sample.unique())
samples

['Patient_A1', 'Patient_A2', 'Patient_A3', 'Patient_A4', 'Patient_G1']

**Create list of  CNVs that correpsonds to unique samples for Zip command**

In [5]:
cnvs=['CNV_'+str(i+1) for i in range(len(samples))]
cnvs

['CNV_1', 'CNV_2', 'CNV_3', 'CNV_4', 'CNV_5']

**Read the mutation file**

In [6]:
mut=pd.read_excel(Patient+"_table.xlsx",index_col=None)
mut["key2"]=mut.Key.str.split(":").apply(lambda x :x[1])
mut["chromosome"]=mut.key2.str.split(".").apply(lambda x :x[0])
mut["position"]=mut.Key.str.split(".").apply(lambda x :x[1])
mut['position'] = mut['position'].astype(float)
mut["chromosome"]=mut.chromosome.str.replace("chr","")
len(mut)

36

**Add  copy number of each sample in last column in loop**

In [7]:
for s,c in zip(samples,cnvs):
    c= cnv[cnv.Sample==s]   # selects each sample
    c=c[['Chromosome', 'Start', 'End','abscopy']]
    c.columns=['chromosome', 'start', 'end', s.split('_')[1]] # strips the full sample name to A1 etc
    c["chromosome"]=c.chromosome.str.replace("chr","")
    c_mut=mut.merge(c,on="chromosome", how="outer")
    c_mut['position'] = c_mut['position'].astype(float)
    c_mut=c_mut[c_mut["position"].between(c_mut.pop("start"),c_mut.pop("end"))]
    c_mut_select=c_mut[["chromosome","position",s.split('_')[1]]]
    mut=mut.merge(c_mut_select,how="left") # fiel is renamed as mut so it goes in loop with previous addition

mut.to_excel(Patient+"_copyadded_output.xlsx",index=None)
len(mut)

36